In [ ]:
import tensorflow as tf
from tensorflow.python.keras import regularizers
import pandas as pd
from tensorflow.python.ops import nn
import numpy as np
from utils import *
from models import *

if tf.__version__ != "2.0.0":
    tf.enable_eager_execution()
"""
    Hyper-parameters
"""
class HyperParameters:
    """
    Easy to manage HyperParameters
    """
    n_epochs = 4
    n_batches = 1000
    target_loss = 1e-5
    thresh_hold = 1e-5
    dataset = "cifar10"
    layer_type = "cnn"
    seed = 1234

params = HyperParameters

"""
    Load data
"""
tf.random.set_seed(params.seed)
x_train, y_train, x_test, y_test= get_data(params.dataset)
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
if params.dataset == "mnist" and params.layer_type == "dense":
    x_train = tf.reshape(x_train,(len(x_train),28*28))
    x_test = tf.reshape(x_test,(len(x_test),28*28))
    n_inputs = 28*28
    n_outputs = 10
    n_hiddens = 784
elif params.dataset in ["cifar10","cifar100"] and params.layer_type == "dense":
    x_train = tf.reshape(x_train,(x_train.shape[0], 32 * 32 * 3))
    x_test = tf.reshape(x_test,(x_test.shape[0], 32 * 32 * 3))
    n_inputs = 32*32*3
    n_outputs = 10
    n_hiddens = 32*32*3
elif params.dataset in ["cifar10","mnist"] and params.layer_type == "cnn":
    n_filters = 64
    n_kernels = 3
    n_outputs = 10
    inp_shape = x_train.shape[1:]
elif params.dataset =="cifar100" and params.layer_type == "cnn":
    n_filters = 64
    n_kernels = 3
    n_outputs = 100
    inp_shape = x_train.shape[1:]
    
train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(
    8192, seed = params.seed).batch(params.n_batches)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test)).shuffle(
    8192, seed = params.seed).batch(params.n_batches)
"""
    Set model
"""
if params.layer_type == "dense":
    model = DenseModel(n_hiddens,n_inputs,n_outputs)
elif params.layer_type == "cnn":
    model = CNNModel(n_filters,n_kernels,n_outputs,inp_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
"""
    Train data
"""
epoch = 0
while (epoch < params.n_epochs) or (epoch_loss_avg.result() < params.target_loss):
    epoch += 1
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_acc_avg = tf.keras.metrics.Mean()
    for x, y in train_dataset:
        with tf.GradientTape() as tape:
            out = model(x)
            batch_loss = my_loss(out, y, n_outputs)
            variables = model.trainable_variables
            grads = tape.gradient(batch_loss, variables)
            optimizer.apply_gradients(zip(grads, variables))
        epoch_loss_avg(batch_loss)
    if epoch%1 == 0:
        print(model.summary())
    for x, y in test_dataset:
        test_out = model(x)
        accuracy = cal_acc(test_out, y)
        epoch_acc_avg(accuracy)
    model.sparsify_weights(params.thresh_hold)
    model.add_layer()
    print('Epoch : {} ----- Loss : {} ----- Acc : {}'.format(epoch, epoch_loss_avg.result(), epoch_acc_avg.result()))

Model: "cnn_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Conv2D)           multiple                  1792      
_________________________________________________________________
cnn_output (Conv2D)          multiple                  1731      
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
classify (MyDenseLayer)      multiple                  30730     
Total params: 34,253
Trainable params: 34,253
Non-trainable params: 0
_________________________________________________________________
None
Epoch : 1 ----- Loss : 5.793919086456299 ----- Acc : 0.18240000307559967
Model: "cnn_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cnn_input (Conv2D)

In [2]:
help(CNNModel)

NameError: name 'CNNModel' is not defined

In [23]:
tf.math.in_top_k(tf.cast(tf.squeeze(y_test),tf.int64),tf.cast(test_out, tf.float32),  1)

<tf.Tensor: id=5974, shape=(10000,), dtype=bool, numpy=array([False, False, False, ..., False, False, False])>

In [16]:
x_train, y_train, x_test, y_test= get_data(params.dataset)
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
model = models.Sequential()
model.add(layers.Conv1D(64, (3), activation='relu', input_shape=input_shape, padding = 'same'))
model.add(layers.Conv1D(64, (3), activation='relu',padding = 'same'))
model.add(layers.Conv1D(64, (3), activation='relu', padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [17]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 28, 64)            5440      
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 28, 64)            12352     
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 28, 64)            12352     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                114752    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                650       
Total params: 145,546
Trainable params: 145,546
Non-trainable params: 0
________________________________________________

In [15]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 11s 191us/sample - loss: 0.1715 - accuracy: 0.9482 - val_loss: 0.0737 - val_accuracy: 0.9749
Epoch 2/10
60000/60000 [==============================] - 11s 178us/sample - loss: 0.0612 - accuracy: 0.9809 - val_loss: 0.0496 - val_accuracy: 0.9831
Epoch 3/10
60000/60000 [==============================] - 10s 175us/sample - loss: 0.0441 - accuracy: 0.9861 - val_loss: 0.0493 - val_accuracy: 0.9853
Epoch 4/10
60000/60000 [==============================] - 10s 166us/sample - loss: 0.0359 - accuracy: 0.9885 - val_loss: 0.0391 - val_accuracy: 0.9885
Epoch 5/10
60000/60000 [==============================] - 10s 168us/sample - loss: 0.0269 - accuracy: 0.9911 - val_loss: 0.0331 - val_accuracy: 0.9904
Epoch 6/10
60000/60000 [==============================] - 10s 174us/sample - loss: 0.0243 - accuracy: 0.9921 - val_loss: 0.0443 - val_accuracy: 0.9871
Epoch 7/10
60000/60000 [====================

In [2]:
from utils import *
x_train, y_train, x_test, y_test= get_data('cifar10')
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]

In [3]:
x_test

<tf.Tensor: id=3, shape=(10000, 32, 32, 3), dtype=float32, numpy=
array([[[[0.61960787, 0.4392157 , 0.19215687],
         [0.62352943, 0.43529412, 0.18431373],
         [0.64705884, 0.45490196, 0.2       ],
         ...,
         [0.5372549 , 0.37254903, 0.14117648],
         [0.49411765, 0.35686275, 0.14117648],
         [0.45490196, 0.33333334, 0.12941177]],

        [[0.59607846, 0.4392157 , 0.2       ],
         [0.5921569 , 0.43137255, 0.15686275],
         [0.62352943, 0.44705883, 0.1764706 ],
         ...,
         [0.53333336, 0.37254903, 0.12156863],
         [0.49019608, 0.35686275, 0.1254902 ],
         [0.46666667, 0.34509805, 0.13333334]],

        [[0.5921569 , 0.43137255, 0.18431373],
         [0.5921569 , 0.42745098, 0.12941177],
         [0.61960787, 0.43529412, 0.14117648],
         ...,
         [0.54509807, 0.38431373, 0.13333334],
         [0.50980395, 0.37254903, 0.13333334],
         [0.47058824, 0.34901962, 0.12941177]],

        ...,

        [[0.26666668, 0.48

In [33]:
input_shape[1]

32

In [34]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3,3), activation= None, input_shape=input_shape, padding = 'same'))
model.add(layers.Conv2D(64, (3,3), activation='relu',padding = 'same'))
model.add(layers.Conv2D(64, (3,3), activation='relu', padding = 'same'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)        36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                1048592   
_________________________________________________________________
dense_9 (Dense)              (None, 10)                170       
Total params: 1,124,410
Trainable params: 1,124,410
Non-trainable params: 0
____________________________________________

In [23]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=10, 
                    validation_data=(x_test, y_test))


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
 4320/50000 [=>............................] - ETA: 2:51 - loss: 2.1192 - accuracy: 0.2309

KeyboardInterrupt: 